In [99]:
import boto3
import pickle
import pandas as pd
from io import StringIO
import io

!pip install gensim
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans;
#from sklearn.neighbors import KDTree
import numpy as np

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [46]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/inroads_vectorized.csv')
inroads_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8')

inroads_vectorized.shape

(10061, 303)

In [88]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/old_vectorized.csv')
old_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

old_vectorized.shape

(941, 302)

In [89]:
old_vectorized.columns

Index(['token', 'vector', '0', '1', '2', '3', '4', '5', '6', '7',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=302)

In [90]:
## extract details

In [91]:
old_vocab = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i].token
    old_vocab.append(test)

old_vocab_df = pd.DataFrame(old_vocab, columns=["token"])

In [92]:
old_vectors_2 = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i].vector
    old_vectors_2.append(test)

In [93]:
import numpy as np
old_array = np.array(old_vectors_2)

In [95]:
old_vectors_df_2 = old_vectorized.drop(['token','vector'], axis=1)
old_vectors_df_2.head

<bound method NDFrame.head of             0         1         2         3         4         5         6  \
0    0.009340 -0.011540 -0.005011  0.170543 -0.070042 -0.022673  0.035479   
1    0.073121 -0.047603  0.091147  0.162190 -0.052149 -0.108441 -0.028311   
2    0.013254  0.050222 -0.089445  0.083483 -0.044747 -0.065899  0.019941   
3    0.017232 -0.138262 -0.069550  0.161501 -0.024385 -0.121781 -0.035268   
4   -0.056970  0.020302 -0.024677 -0.048865  0.064803 -0.033035 -0.029016   
..        ...       ...       ...       ...       ...       ...       ...   
936 -0.021196 -0.011311 -0.073023  0.030679  0.117935  0.040947  0.024909   
937 -0.028905  0.093232 -0.058349 -0.013379 -0.086675  0.104365  0.064961   
938 -0.074280  0.093354 -0.145046  0.006174  0.000712  0.073791  0.088202   
939 -0.045976 -0.006391 -0.056196 -0.057425 -0.018857 -0.025275  0.079501   
940  0.014605  0.167768 -0.082776 -0.017683 -0.045675  0.065280  0.051255   

            7         8         9  ...       

In [96]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering, kmeans_clustering.cluster_centers_, idx;

In [97]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df, tree;

In [100]:
kmodel,centers,clusters = clustering_on_wordvecs(old_vectors_2, 9);
centroid_map = dict(zip(old_vocab, clusters));

ValueError: could not convert string to float: '[ 9.34010372e-03 -1.15401922e-02 -5.01083815e-03  1.70543134e-01\n -7.00417161e-02 -2.26731710e-02  3.54789868e-02 -3.35189812e-02\n  3.82505096e-02  4.57449667e-02 -1.86700057e-02 -5.73066249e-02\n -9.70353857e-02  8.72526467e-02 -1.87249873e-02  1.17057292e-02\n  3.54982428e-02  1.08581044e-01 -1.13191493e-01  7.59809762e-02\n  7.05875158e-02 -8.44866931e-02 -1.27818482e-02 -6.16002679e-02\n  4.38925577e-03 -1.22045800e-02  7.72628710e-02  1.24636769e-01\n  6.59519881e-02  2.50583980e-02  8.72580707e-02 -3.28506380e-02\n -1.36340475e-02  6.65055439e-02  4.84936908e-02 -2.44322605e-02\n -3.68555337e-02 -2.49186028e-02  4.41841893e-02 -1.03854202e-01\n -4.31859568e-02  5.83787784e-02  8.61320421e-02 -2.74619665e-02\n  2.24119574e-02  4.63581495e-02 -4.00820263e-02 -6.64951578e-02\n  2.62899250e-02  3.79224271e-02 -2.43805442e-02  2.30064411e-02\n -8.43966305e-02  3.19873579e-02  6.24968074e-02  3.38818692e-02\n -6.13099188e-02  4.67965053

In [84]:
predicted_cluster = {}

for i in range(0, len(inroads_vectors_2)):
    word = inroads_vocab[i]
    temp = kmodel.predict(inroads_vectors_2[i].reshape(1,-1))[0]
    predicted_cluster.update({word:temp})

In [93]:
predicted_cluster

{'research': 0,
 'article': 0,
 'personality': 0,
 'social': 5,
 'psychology': 0,
 'benevolent': 5,
 'sexism': 6,
 'attitudes': 0,
 'toward': 5,
 'bulletin': 4,
 'society': 5,
 'motherhood': 0,
 'reproductive': 6,
 'right': 5,
 'reprint': 0,
 'multi': 0,
 'study': 0,
 'longitudinal': 1,
 'examination': 0,
 'abortion': 6,
 'huang': 7,
 'paul': 3,
 'davies': 3,
 'chris': 7,
 'sibley': 4,
 'danny': 3,
 'osborne': 2,
 'abstract': 0,
 'although': 8,
 'ideology': 5,
 'that': 2,
 'highly': 0,
 'revere': 3,
 'women': 6,
 'conform': 2,
 'traditional': 0,
 'gender': 0,
 'cloak': 7,
 'superficially': 2,
 'positive': 2,
 'tone': 0,
 'place': 0,
 'upon': 2,
 'pedestal': 7,
 'inherently': 5,
 'restrictive': 6,
 'accordingly': 0,
 'because': 2,
 'paternalistic': 6,
 'beliefs': 2,
 'associate': 0,
 'with': 2,
 'base': 5,
 'idealization': 2,
 'roles': 0,
 'which': 0,
 'include': 0,
 'should': 2,
 'predict': 2,
 'people': 2,
 'data': 1,
 'from': 8,
 'nationwide': 5,
 'panel': 0,
 'show': 0,
 'hostile': 